In [1]:
# импорт библиотек
import os
import cv2
import re
import copy
import datetime
import pandas as pd
import numpy as np

from PIL import Image as PILImage

from pytesseract import Output
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

from img2table.document import Image as DocumentImage
from img2table.ocr import TesseractOCR

from IPython.display import display_html



class Form405:
    '''
    Класс для распознавания отметки о взятии крови из "Учетной карточки донора" (Форма № 405-05/у).
    Атрибуты класса:    
        folder                         директория с файлами
        image_file                     название исходного файла изображения с расширением
        image_path                     путь к исходному файлу изображения           
        person_id                      префикс названия файлов, соответствующих определенному пользователю
        csv_path                       путь к исходному csv-файлу
        csv_recognised_path            путь к сгенерированному csv-файлу
        image_original                 исходный изображенное в обработке
        image_processed                преобразованное изображение в обработке
        image_processed_path           путь сохранения преобразованного изображения
        document_image                 объекта рисунка для разметки таблицы
        table_coords                   упорядоченный словарь с координатами ячеек
        n_rows                         число строк в таблице
        n_cols                         число столбцов в таблице
        x1, y1, x2, y2                 координаты левого верхнего и правого нижнего углов таблицы
        avr_cell_width                 расчетная средняя ширина ячеек
        avr_cell_height                расчетная средняя высота ячеек
        flag_split_cell                флаг расщепления ячеек
        flag_size_incorrected          флаг некорректного размера
        num_field                      число основных полей в таблице
        df_original                    исходный датафрейм
        df_recognised                  сгенерированный датафрейм
        accuracy                       точность распознавания
    
   Последовательность операция для запуска распознавания:
       1. Создание экземпляра класса и запуск предобработки
            obj = Form405('405/', '245365 .jpg')
       2. Получение разметки и параметров для таблицы
            obj.get_table_coords()
       3. Распознавание и сохранение файла с результатами
            obj.get_dataframe()
    '''

    # инициализация класса и назначение пути к файлу изображения
    def __init__(self, folder, file):
        # иинициализация путей
        self.folder = folder
        self.image_file = file
        self.image_path = self.folder + self.image_file        
        self.person_id = re.match(r'(.*)(\s+\..*)', file).group(1)
        self.csv_path = self.folder + self.person_id + '.csv'
        self.csv_recognised_path = self.folder + self.person_id + '_recognised.csv'
        # изапуск предобработки
        self.make_preprocessing()
        
                
         
    # функция предобработки изображения
    def make_preprocessing(self):
        self.image_original = cv2.imread(self.image_path)    
        # обрезка верхней части изображения
        height = int(self.image_original.shape[0])
        half_height = int(self.image_original.shape[0] / 2.)
        self.image_processed = self.image_original[half_height:height, :]
        # конвертирование цвета изобраения в оотенки серого
        self.image_processed = cv2.cvtColor(self.image_processed, cv2.COLOR_BGR2GRAY)
        # добавление размытия для удаления шума
        self.image_processed = cv2.medianBlur(self.image_processed, 1)
        # введение порогового значения для сегментации изобраения
        self.image_processed = cv2.adaptiveThreshold(
            self.image_processed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 11
        )
        # сохранение файла
        self.image_processed_path = self.folder + self.person_id + '_processed.png'
        cv2.imwrite(self.image_processed_path, self.image_processed)

    

    # функция определения скруктуры таблицы
    def __get_table_pos(self, table_extracted):
        # число элементов в таблице
        self.n_rows = len(table_extracted)
        self.n_cols = len(table_extracted[0])
        # координаты левого верхнего и правого нижнего углов таблицы
        self.x1 = table_extracted[0][0].bbox.x1
        self.y1 = table_extracted[0][0].bbox.y1
        self.x2 = table_extracted[self.n_rows-1][-1].bbox.x2
        self.y2 = table_extracted[self.n_rows-1][-1].bbox.y2
        # средний размер ячейки
        self.avr_cell_width = int((self.x2 - self.x1) / self.n_cols)
        self.avr_cell_height = int((self.y2 - self.y1) / self.n_rows)
        # проверка на сплит-ячейки
        self.flag_split_cell = False
        self.flag_size_incorrected = False
        for i in range(self.n_rows):
            if len(table_extracted[i]) != self.n_cols:
                self.flag_split_cols = True
        for i in range(self.n_rows):
            # проверка сопадения числа ячеек в строке
            if len(table_extracted[i]) != self.n_cols:
                    self.flag_split_cell = True
            # проверка размера ячеек
            for j in range(self.n_cols):
                current_cell_width = int(table_extracted[i][j].bbox.x2 - table_extracted[i][j].bbox.x1)
                current_cell_height = int(table_extracted[i][j].bbox.y2 - table_extracted[i][j].bbox.y1)
                if (
                    (abs(self.avr_cell_width - current_cell_width) > 0.1 * self.avr_cell_width)
                    or (abs(self.avr_cell_height - current_cell_height) > 0.1 * self.avr_cell_height)
                ):
                    self.flag_size_incorrected = True
    
    
 
    # функция получения таблицы с координатами ячеек
    def get_table_coords(self):
        '''
        Метод extract_tables некорректно распознаёт текст. записанный в несколько строк в шапке таблицы. 
        После чего он добавляет повторяющиеся соседние ячейки в столбцы. Такие дубли необходимо удалить.
        '''
        # (1) получение разметки таблицы
        # создания объекта рисунка типа Image для разметки таблицы
        self.document_image = DocumentImage(src=self.image_processed_path )
        # распознавание таблицы с ячейками
        table_coords = self.document_image.extract_tables(ocr=None, 
                                                          implicit_rows=False,
                                                          borderless_tables=False,
                                                          min_confidence=50)[0].content

        # расположение таблицы, число ячеек и расчетный размер ячейки
        table_params = self.__get_table_pos(table_coords)

        # (2) исправление разметки таблицы
        table_coords_copy = copy.deepcopy(table_coords)   
        # проход по строкам словаря разметки    
        col_size_error = False
        for i in range(len(table_coords)):
            # удаление дублирующихся ячеек в столбцах
            shrinked_list = []
            last_cell = None
            last_cell_reserve = None
            next_cell = None
            for j in range(len(table_coords_copy[i])):
                last_cell = next_cell
                next_cell = table_coords_copy[i][j]
                # первая запись в списке заносится отдельно
                if j == 0:
                    last_cell_reserve = next_cell
                    shrinked_list.append(next_cell)                
                    continue            
                # коррекция ширины текущей ячейки за счет объединения со следующей
                if (j < len(table_coords[0]) - 1) and (next_cell.bbox.x2 - next_cell.bbox.x1 < 0.9*self.avr_cell_width):
                        table_coords_copy[i][j+1].bbox.x1 = next_cell.bbox.x1
                        # восстановление предыщей записи из копии
                        last_cell = last_cell_reserve
                        continue
                # сравнение координат соседних ячеек
                if j > 0 and ((next_cell.bbox.x1 != last_cell.bbox.x1)
                              or (next_cell.bbox.x2 != last_cell.bbox.x2)
                              or (next_cell.bbox.y1 != last_cell.bbox.y1)
                              or (next_cell.bbox.y2 != last_cell.bbox.y2)
                ):
                    shrinked_list.append(next_cell)
                    last_cell_reserve = next_cell
            table_coords_copy[i] = shrinked_list
            # проверка совпадения размеров столбцов
            if i > 0 and col_size_error == False and len(table_coords_copy[i]) != len(table_coords_copy[i-1]):
                col_size_error = True
                print('Обнаружено несовпадение размеров столбцов при разметке таблицы!')        
        # уточнение параметров разметки
        self.__get_table_pos(table_coords_copy)
        # сохранение OrderedDict с координатами в строках
        self.table_coords = table_coords_copy
    
    
  
    # функция получения координат ячеек для конкретного столбца\строки
    def get_row_coords(self, num_row, col_coords=True):    
        row_coords = []
        # получениe координат ячеек для столбца
        if col_coords == True:
            for i in range(len(self.table_coords)):
                row_coords.append(self.table_coords[i][num_row].bbox)
        # получениe координат ячеек для строки
        else:
            for cell in self.table_coords[num_row]:
                row_coords.append(cell.bbox)
        return row_coords
    
    

    # функция коррекции записи для типа донорства
    def __improve_type(self, stroke):
        try:
            stroke_improved = re.sub('3|8', 'в', stroke)
            stroke_improved = re.sub('5|6', 'б', stroke_improved)
            finded = re.match(r'.*((?:кр|пл|ц)(?:/д)|(?:т/ф))(.*)((?:\(бв\))|(?:\(пл\)))', stroke_improved)
            return finded.group(1) + ' ' + finded.group(3)
        except:
            return "UNKNOWN"
    
    

    # функция оценки адекватности распознования
    def __check_adequacy(self, stroke, col_type=0):
        # оценка количества
        if col_type == 'quantity':
            try:
                # проверка нахождения количества сданной крови в пределах от 50 до 700 мл
                if 50 <= int(stroke) <= 700:
                    return True, stroke
                else:
                    return False, "UNKNOWN"
            except:
                return False, "UNKNOWN"
        # оценка даты
        elif col_type == 'date':
            # дата приказа о форме справки 
            init_date = datetime.datetime.strptime(r'31.03.2005', '%d.%m.%Y')
            # текущее время
            current_date = datetime.datetime.now()
            date_stroke = ''
            # перебор возможных вариантов записи
            for pattern in [r'%d.%m.%Y', r'%d.%m.%y', r'%d-%m-%Y', r'%d-%m-%y']:
                try:
                    date_stroke = datetime.datetime.strptime(stroke, pattern)
                    break
                except:
                    pass
            # проверка нахождения дат в пределах от даты приказа до текущей
            if date_stroke != '' and (init_date <= date_stroke <= current_date):
                # вывод унифицированного варианта записи напр. 31.12.2020
                return True, date_stroke.strftime('%d.%m.%Y')
            else:
                return False, "UNKNOWN"
        # коррекция записи для типа донорства 
        elif col_type == 'type':
            stroke_corrected = self.__improve_type(stroke)
            if stroke_corrected != "UNKNOWN":
                return True, stroke_corrected
            else:
                return False, "UNKNOWN"
        # вывод без проверки    
        else:
            return True, stroke
    
    

    # функция распознавания содержимого ячеек 
    def recognize_cell(self,
                       cell_coords, 
                       col_type=0,
                       ocr_config=r'--psm 6 --oem 3', 
                       re_mask=re.match(r'(.*)( \..*)', '245365 .jpg').group(1), 
                       prc_shrink=0):
        '''
        Данные об области распознавания подаются в виде списка cell_coords в структурах координат Bbox(x1, y1, x2, y2). 
        Для разных типов данных стоит использовать разные конфигурации. 
        Например, для целых чисел r"--psm 7 --oem 3 -c tessedit_char_whitelist=0123456789",
        для даты - r"--psm 13 --oem 1 -c tessedit_char_whitelist=.-0123456789". 
        Подробно о конфигурации можно узнать вызовом !tesseract --help-extra.
        Полный перечень команд конфигурирования вызовом !tesseract --print-parameters.
        Для очистки данных можно подать исполняемую строку регэкспов вида: r"re.match(('.*'), recognized).group(0)".
        Такая строка для целых чисел r"re.match(r'(0*)(\d{2,3})', recognized).group(2)",
        для дат - r"re.match(r'(.*)((?:\d{2})(?:\.|-)(?:\d{2})(?:\.|-)(?:\d{2,4}))(.*)', recognized).group(2)".
        Перед выдачей значения проверяются на адекватность. Некорректные значения записываются как "UNKNOWN".
        '''
        img = cv2.imread(self.image_processed_path)
        text_recognized = []
        for cell in cell_coords:        
            x1 = cell.x1
            y1 = cell.y1
            x2 = cell.x2
            y2 = cell.y2
            # сужение размера ячейки
            x_shift = abs(int((x2-x1)*prc_shrink/100.))
            y_shift = abs(int((y2-y1)*prc_shrink/100.))
            x1 = x1 + x_shift
            x2 = x2 - x_shift
            y1 = y1 + y_shift
            y2 = y2 - y_shift
            # распознование содержимого ячейки c преобразованием формата координат (x,y) в формат инднксов [y,x]
            recognized = pytesseract.image_to_string(img[y1:y2, x1:x2], lang='rus', config=ocr_config).strip()
            # очиска строки
            finded = recognized
            try:
                finded = eval(re_mask)
            except:
                pass
            # проверка адекватности
            _, finded = self.__check_adequacy(finded, col_type)
            text_recognized.append(finded)
        return text_recognized
    
    

    # функция проверки типа столбца
    def __check_cell_type(self, stroke, type_expected):
        # проверка преобразования строки в целое число
        if type_expected == 'quantity':
            try:
                int(stroke)
                return True
            except:
                return False
        # проверка преобразования строки в дату
        elif type_expected == 'date':
            try:
                datetime.datetime.strptime(stroke, '%d.%m.%Y')
                return True
            except:
                return False
        # проверка наличия в строке кириллических букв
        elif type_expected == 'type':
            try:
                re.match(r'^[а-я]{1}', stroke).group()
                return True
            except:
                return False  
        return None
    

    # функция построение датафрейма из распознанных данных
    def get_dataframe(self):
        '''
        Построение датафрейма из распознанных данных возможно при корректной разметке таблицы в документе. 
        Если она не корректна - датафрейм не формируется. 
        Каждая запись о заборе крови состоит из трех упорядоченных полей, иногда к ним добавляется поле с подписью.
        В выходном датафрейме записи добавляются вниз таблицы из трех полей. 
        '''
        # проверка корректности разметки
        if (self.flag_split_cell and self.flag_size_incorrected) == True:
            print('Таблица размечена не корректно! Распознавание не проведено.')
            return None
        else:
            # определение числа полей
            self.num_fields = 0
            if self.n_cols%4 == 0:
                self.num_fields = 4
            elif self.n_cols%3 == 0:
                self.num_fields = 3    
            else:
                print('Неизвестный формат таблицы! Распознавние не проведено.')
                return        
            # параметры для разных типов данных
            data_type = ['date', 'type', 'quantity']
            data_config = [r'--psm 13 --oem 1 -c tessedit_char_whitelist=.-0123456789', 
                           r'--psm 7 --oem 3',
                           r'--psm 7 --oem 3 -c tessedit_char_whitelist=0123456789']
            data_re = [r"re.match(r'(.*)((?:\d{2})(?:\.|-)(?:\d{2})(?:\.|-)(?:\d{2,4}))(.*)', recognized).group(2)", 
                       r"re.match(('.*'), recognized).group(0)", 
                       r"re.match(r'(0*)(\d{2,3})', recognized).group(2)"]
            df_recognised = pd.DataFrame({'date':[],'type':[],'quantity':[]})
            data_dict = {}
            for i in range(self.n_cols):
                # текущий тип поля
                current_field = (i + self.num_fields) % self.num_fields
                # пропуск поля с подписью
                if self.num_fields == 4 or current_field == 3:
                    continue
                # получение координат ячеек
                row_coords = self.get_row_coords(i, col_coords=True)
                # распознование по столбцам
                text_recognized = self.recognize_cell(row_coords,
                                                 col_type=data_type[current_field],
                                                 ocr_config=data_config[current_field],
                                                 re_mask=data_re[current_field],
                                                 prc_shrink=5)
                # подтверждение типа данных от некоторых ячеек
                if sum([self.__check_cell_type(stroke, data_type[current_field]) for stroke in text_recognized]) > 0:
                    # запись текущего столбца по текущему ключу во временный словарь
                    data_type_confirmed = data_type[current_field]
                    data_dict[data_type_confirmed] = text_recognized
                    # завершение формирования блока записей
                    if current_field == 2:                    
                        df_added = pd.DataFrame(data_dict)[2:]
                        # удаление пустой строки в конце блока
                        if current_field > 0:
                            if sum(list(map(lambda x: x == 'UNKNOWN', df_added.iloc[-1]))) == self.num_fields:
                                df_added = df_added.iloc[:-1]
                        # добавление блока в общий датафрйем
                        df_recognised = pd.concat([df_recognised, df_added], axis=0)
                        # очистка словаря
                        data_dict = {}
                        df_recognised.reset_index(drop=True, inplace=True)
            # сохрания результатов распознавания в csv-файл
            df_recognised.to_csv(self.csv_recognised_path)
            self.df_recognised = df_recognised
    
    

    def estimate_accuracy(self):
        self.df_original = pd.read_csv(self.csv_path)
        # проверка контрольного файла 
        try:
            # проверка вхождения идентификаторв в название файла
            if str(self.df_original['ID пользователя'][0]) in self.csv_path:
                self.accuracy = (self.df_recognised['date'] == self.df_original['Дата донации']).mean()
                print(f'Значение Accuracy по распознавании даты = %.2f' % self.accuracy)
        except:
            print('ID пользователя не совпадает с названием файла данных!')
        return self.accuracy

__Генерация результатов__  
Попробуем применить класс на качественном изображении. Ниже последовательность вызовов для генерации файла результатов распознавания.

In [2]:
# (1) создание экземпляра класса и запуск предобработки
doc = Form405('405/', '245365 .jpg')
# (2) получение разметки и параметров для таблицы
doc.get_table_coords()
# (3) распознавание и сохранение файла с результатами
doc.get_dataframe()

# расчет точности распознавание по полю даты
doc.estimate_accuracy()
doc.df_recognised

Значение Accuracy по распознавании даты = 1.00


,date,type,quantity
0,14.01.2009,кр/д (бв),370
1,14.07.2009,кр/д (бв),450
2,25.01.2010,кр/д (бв),450
3,07.02.2011,кр/д (бв),450
4,08.08.2011,кр/д (бв),450
5,29.02.2012,пл/д (бв),250
6,21.11.2016,ц/д (бв),200
7,23.12.2016,пл/д (бв),270
8,24.01.2017,ц/д (бв),217
9,27.02.2017,пл/д (бв),260
